## Working with Transformers in the HuggingFace Ecosystem

In this laboratory exercise we will learn how to work with the HuggingFace ecosystem to adapt models to new tasks. As you will see, much of what is required is *investigation* into the inner-workings of the HuggingFace abstractions. With a little work, a little trial-and-error, it is fairly easy to get a working adaptation pipeline up and running.

### Exercise 1: Sentiment Analysis (warm up)

In this first exercise we will start from a pre-trained BERT transformer and build up a model able to perform text sentiment analysis. Transformers are complex beasts, so we will build up our pipeline in several explorative and incremental steps.

#### Exercise 1.1: Dataset Splits and Pre-trained model
There are a many sentiment analysis datasets, but we will use one of the smallest ones available: the [Cornell Rotten Tomatoes movie review dataset](cornell-movie-review-data/rotten_tomatoes), which consists of 5,331 positive and 5,331 negative processed sentences from the Rotten Tomatoes movie reviews.

**Your first task**: Load the dataset and figure out what splits are available and how to get them. Spend some time exploring the dataset to see how it is organized. Note that we will be using the [HuggingFace Datasets](https://huggingface.co/docs/datasets/en/index) library for downloading, accessing, splitting, and batching data for training and evaluation.

In [1]:
from datasets import load_dataset

# Carico i tre split del dataset Rotten Tomatoes 
train_set = load_dataset("cornell-movie-review-data/rotten_tomatoes", split="train")
val_set = load_dataset("cornell-movie-review-data/rotten_tomatoes", split="validation")
test_set = load_dataset("cornell-movie-review-data/rotten_tomatoes", split="test")


In [2]:
# Numero di esempi in ciascuno split
print(f"Train-set: {len(train_set)}")
print(f"Validation-set: {len(val_set)}")
print(f"Test-set: {len(test_set)}")

Train-set: 8530
Validation-set: 1066
Test-set: 1066


#### Exercise 1.2: A Pre-trained BERT and Tokenizer

The model we will use is a *very* small BERT transformer called [Distilbert](https://huggingface.co/distilbert/distilbert-base-uncased) this model was trained (using self-supervised learning) on the same corpus as BERT but using the full BERT base model as a *teacher*.

**Your next task**: Load the Distilbert model and corresponding tokenizer. Use the tokenizer on a few samples from the dataset and pass the tokens through the model to see what outputs are provided. I suggest you use the [`AutoModel`](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html) class (and the `from_pretrained()` method) to load the model and `AutoTokenizer` to load the tokenizer).

In [3]:
from transformers import AutoTokenizer, AutoModel
import torch

model_name = "distilbert-base-uncased"

model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Estraggo i primi 5 esempi dal train-set
texts = train_set["text"][:5]

inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# Valuto l'output del modello
model.eval()
with torch.no_grad():
    outputs = model(**inputs)

print(outputs.last_hidden_state.shape) #Output = Embedding di forma [5, 52, 768]

torch.Size([5, 52, 768])


#### Exercise 1.3: A Stable Baseline

In this exercise I want you to:
1. Use Distilbert as a *feature extractor* to extract representations of the text strings from the dataset splits;
2. Train a classifier (your choice, by an SVM from Scikit-learn is an easy choice).
3. Evaluate performance on the validation and test splits.

These results are our *stable baseline* -- the **starting** point on which we will (hopefully) improve in the next exercise.

**Hint**: There are a number of ways to implement the feature extractor, but probably the best is to use a [feature extraction `pipeline`](https://huggingface.co/tasks/feature-extraction). You will need to interpret the output of the pipeline and extract only the `[CLS]` token from the *last* transformer layer. *How can you figure out which output that is?*

In [4]:
from transformers import pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import numpy as np
from tqdm import tqdm


def extract_features(dataset, extractor):
    all_features = []
    all_labels = []
    
    for data in tqdm(dataset):
        text = data["text"]
        label = data["label"]
        
        with torch.no_grad():
            features = extractor(text, truncation=True, padding=True, max_length=256)
            embedding_cls = features[0][0]
        
        all_features.append(embedding_cls)
        all_labels.append(label)
    
    return all_features, all_labels

# Definizione della pipeline per l'estrazione delle features
feature_extractor = pipeline("feature-extraction", model="distilbert-base-uncased", tokenizer="distilbert-base-uncased", device=0)

# Estrazione dei CLS embedding per ciascuno split del dataset
X_train, y_train = extract_features(train_set, feature_extractor)
X_val, y_val = extract_features(val_set, feature_extractor)
X_test , y_test = extract_features(test_set, feature_extractor)

# Normalizzazione delle features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)


# Support Vector Classifier per la testa di classificazione
svm = SVC(kernel='linear')

svm.fit(X_train_scaled, y_train)

# Predizione sul validation set
y_pred = svm.predict(X_val_scaled)

val_acc = accuracy_score(y_val, y_pred)

print(f"Validation Accuracy: {val_acc}") # Validation Accuracy: 0.8096

# Predizione sul test set
y_pred_test = svm.predict(X_test_scaled)

test_acc = accuracy_score(y_test, y_pred_test)

print(f"Test Accuracy: {test_acc}") # Test Accuracy: 0.7908




Device set to use cuda:0
100%|██████████| 1066/1066 [00:14<00:00, 72.67it/s]


Validation Accuracy: 0.8095684803001876
Test Accuracy: 0.7908067542213884


### Risultati Baseline: Accuracy su Validation e Test Set

- **Validation Accuracy:** 0.8096
- **Test Accuracy:** 0.7908

Questi risultati rappresentano le performance ottenute utilizzando **DistilBERT** come feature extractor e un **classificatore SVM lineare** come baseline per il task di sentiment analysis sul dataset **Rotten Tomatoes**.


-----
### Exercise 2: Fine-tuning Distilbert

In this exercise we will fine-tune the Distilbert model to (hopefully) improve sentiment analysis performance.

#### Exercise 2.1: Token Preprocessing

The first thing we need to do is *tokenize* our dataset splits. Our current datasets return a dictionary with *strings*, but we want *input token ids* (i.e. the output of the tokenizer). This is easy enough to do my hand, but the HugginFace `Dataset` class provides convenient, efficient, and *lazy* methods. See the documentation for [`Dataset.map`](https://huggingface.co/docs/datasets/v3.5.0/en/package_reference/main_classes#datasets.Dataset.map).

**Tip**: Verify that your new datasets are returning for every element: `text`, `label`, `intput_ids`, and `attention_mask`.

In [5]:
def tokenize_function(example):
    
    encoding = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )
    example["input_ids"] = encoding["input_ids"]
    example["attention_mask"] = encoding["attention_mask"]
    return example

tokenized_train_set = train_set.map(tokenize_function)
tokenized_val_set = val_set.map(tokenize_function)
tokenized_test_set = test_set.map(tokenize_function)

print(tokenized_train_set.features) # {"text", "label", "input_ids", "mask_attention"}

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}


#### Exercise 2.2: Setting up the Model to be Fine-tuned

In this exercise we need to prepare the base Distilbert model for fine-tuning for a *sequence classification task*. This means, at the very least, appending a new, randomly-initialized classification head connected to the `[CLS]` token of the last transformer layer. Luckily, HuggingFace already provides an `AutoModel` for just this type of instantiation: [`AutoModelForSequenceClassification`](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodelforsequenceclassification). You will want you instantiate one of these for fine-tuning.

In [7]:
from transformers import AutoModelForSequenceClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

my_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)
my_model.to(device)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


#### Exercise 2.3: Fine-tuning Distilbert

Finally. In this exercise you should use a HuggingFace [`Trainer`](https://huggingface.co/docs/transformers/main/en/trainer) to fine-tune your model on the Rotten Tomatoes training split. Setting up the trainer will involve (at least):


1. Instantiating a [`DataCollatorWithPadding`](https://huggingface.co/docs/transformers/en/main_classes/data_collator) object which is what *actually* does your batch construction (by padding all sequences to the same length).
2. Writing an *evaluation function* that will measure the classification accuracy. This function takes a single argument which is a tuple containing `(logits, labels)` which you should use to compute classification accuracy (and maybe other metrics like F1 score, precision, recall) and return a `dict` with these metrics.  
3. Instantiating a [`TrainingArguments`](https://huggingface.co/docs/transformers/v4.51.1/en/main_classes/trainer#transformers.TrainingArguments) object using some reasonable defaults.
4. Instantiating a `Trainer` object using your train and validation splits, you data collator, and function to compute performance metrics.
5. Calling `trainer.train()`, waiting, waiting some more, and then calling `trainer.evaluate()` to see how it did.

**Tip**: When prototyping this laboratory I discovered the HuggingFace [Evaluate library](https://huggingface.co/docs/evaluate/en/index) which provides evaluation metrics. However I found it to have insufferable layers of abstraction and getting actual metrics computed. I suggest just using the Scikit-learn metrics...

In [8]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from sklearn.metrics import classification_report, accuracy_score


def evaluate_metrics(predictions_with_labels):
    logits, true_labels = predictions_with_labels
    predicted_labels = np.argmax(logits, axis=-1)

    report = classification_report(true_labels, predicted_labels, output_dict=True, zero_division=0)
    accuracy = accuracy_score(true_labels, predicted_labels)

    return {
        "accuracy": accuracy,
        "precision": report['1']['precision'],
        "recall": report['1']['recall'],
        "f1_score": report['1']['f1-score']
    }

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=my_model,
    args=training_args,
    train_dataset=tokenized_train_set,
    eval_dataset=tokenized_val_set,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics= evaluate_metrics,
)

trainer.train()

results = trainer.evaluate()


C:\Users\dario\AppData\Local\Temp\ipykernel_16120\2289614583.py:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: dariomezzasalma14 (dariomezzasalma14-universit-di-firenze) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 Score
1,No log,0.372216,0.838649,0.798347,0.906191,0.848858
2,0.353900,0.365054,0.843340,0.832727,0.859287,0.845799
3,0.353900,0.429370,0.845216,0.866534,0.816135,0.840580
4,0.152900,0.497874,0.853659,0.882353,0.816135,0.847953
5,0.152900,0.553306,0.845216,0.832130,0.864916,0.848206


In [9]:
trainer.evaluate(tokenized_test_set)

{'eval_loss': 0.4072437286376953,
 'eval_accuracy': 0.8452157598499062,
 'eval_precision': 0.8394833948339483,
 'eval_recall': 0.8536585365853658,
 'eval_f1_score': 0.8465116279069768,
 'eval_runtime': 29.6871,
 'eval_samples_per_second': 35.908,
 'eval_steps_per_second': 1.145,
 'epoch': 5.0}

- **Accuracy sul test set:** 84.52%

### Conclusioni

- Dai risultati ottenuti possiamo osservare che il modello **DistilBERT fine-tuned** ha raggiunto delle performance leggermente migliori rispetto alla baseline, in cui DistilBERT veniva utilizzato solo come feature-extractor e un **SVM lineare** fungeva da testa di classificazione. In particolare, l'accuracy sul test set è passata da circa **79%** (baseline SVM) a circa **84%** (modello fine-tuned).
- E' da notare come già dalla prima epoca si raggiunga una **validation accuracy** del 84% circa e al trascorrere delle epoche la **validation loss** sale mantenendo allo stesso tempo alte le prestazioni. Sono già sufficienti 2-3 epoche di training. 

-----
### Exercise 3: Choose at Least One


#### Exercise 3.1: Efficient Fine-tuning for Sentiment Analysis (easy)

In Exercise 2 we fine-tuned the *entire* Distilbert model on Rotten Tomatoes. This is expensive, even for a small model. Find an *efficient* way to fine-tune Distilbert on the Rotten Tomatoes dataset (or some other dataset).

**Hint**: You could check out the [HuggingFace PEFT library](https://huggingface.co/docs/peft/en/index) for some state-of-the-art approaches that should "just work". How else might you go about making fine-tuning more efficient without having to change your training pipeline from above?

In [17]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "distilbert-base-uncased"

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from peft import LoraConfig, TaskType, get_peft_model

# Configurazione del modello LoRA
peft_config = LoraConfig(
    inference_mode=False,
    r=8, 
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_lin", "v_lin"],
    task_type=TaskType.SEQ_CLS,  # Sequence Classification Task
)

model = get_peft_model(model, peft_config)
model.to(device)

# Quanti parametri apprendibili ci sono nel modello LoRA?
model.print_trainable_parameters()

trainable params: 739,586 || all params: 67,694,596 || trainable%: 1.0925


In [19]:
training_arguments = TrainingArguments(
    output_dir="./lora_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    logging_dir="./lora_logs",
    weight_decay=0.01,
    load_best_model_at_end=True,
)

trainer_lora = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=tokenized_train_set,
    eval_dataset=tokenized_val_set,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=evaluate_metrics,
)

trainer_lora.train()

results_lora = trainer_lora.evaluate()

C:\Users\dario\AppData\Local\Temp\ipykernel_16120\441018047.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_lora = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 Score
1,No log,0.357814,0.839587,0.816434,0.876173,0.845249
2,0.391700,0.349081,0.842402,0.850288,0.831144,0.840607
3,0.391700,0.344017,0.842402,0.839851,0.846154,0.842991
4,0.289900,0.358028,0.839587,0.845420,0.831144,0.838221
5,0.289900,0.369767,0.841463,0.843396,0.838649,0.841016


In [ ]:
# Valutazione del modello LoRA sul test set 
trainer_lora.evaluate(tokenized_test_set)

{'eval_loss': 0.39460161328315735,
 'eval_accuracy': 0.8330206378986866,
 'eval_precision': 0.8268876611418048,
 'eval_recall': 0.8424015009380863,
 'eval_f1_score': 0.8345724907063197,
 'eval_runtime': 31.5639,
 'eval_samples_per_second': 33.773,
 'eval_steps_per_second': 1.077,
 'epoch': 5.0}

- **Test Accuracy:** 83,30% 

### Riepilogo risultati

| Metodo                                 | Test Accuracy (%) |
|-----------------------------------------|-------------------|
| Baseline (DistilBERT + SVM, Esercizio 1.3)      | 79.08            |
| Fine-tuning completo (Esercizio 2.3)            | 84.52            |
| Fine-tuning LoRA (Esercizio 3.1)                | 83.30            |

### Conclusioni

- L'adozione di LoRA (Low-Rank Adaptation) ha permesso il fine-tuning di soltanto una piccola frazione dei parametri del modello (~ 1%), riducendo significativamente i tempi di addestramento.
- Nonostante la riduzione drastica del numero di parametri aggiornati, il modello fine-tunato con LoRA ha raggiunto una **test accuracy** quasi equivalente a quella ottenuta con il fine-tuning completo.
- E' da notare come nell'utilizzo di LoRA la **validation loss** si mantenga pressochè stabile, senza aumentare tanto come nel fine-tuning completo.

#### Exercise 3.2: Fine-tuning a CLIP Model (harder)

Use a (small) CLIP model like [`openai/clip-vit-base-patch16`](https://huggingface.co/openai/clip-vit-base-patch16) and evaluate its zero-shot performance on a small image classification dataset like ImageNette or TinyImageNet. Fine-tune (using a parameter-efficient method!) the CLIP model to see how much improvement you can squeeze out of it.

**Note**: There are several ways to adapt the CLIP model; you could fine-tune the image encoder, the text encoder, or both. Or, you could experiment with prompt learning.

**Tip**: CLIP probably already works very well on ImageNet and ImageNet-like images. For extra fun, look for an image classification dataset with different image types (e.g. *sketches*).

In [4]:
# Your code here.

#### Exercise 3.3: Choose your Own Adventure

There are a *ton* of interesting and fun models on the HuggingFace hub. Pick one that does something interesting and adapt it in some way to a new task. Or, combine two or more models into something more interesting or fun. The sky's the limit.

**Note**: Reach out to me by email or on the Discord if you are unsure about anything.

In [5]:
# Your code here.